In [49]:
from collections import OrderedDict
from torch.optim.lr_scheduler import StepLR
import torch
from torchvision.ops import nms
from torchvision.ops.boxes import box_convert,box_iou
from torchvision.models.detection import fasterrcnn_resnet50_fpn_v2 
from torchvision.models.detection import fasterrcnn_resnet50_fpn 
import numpy as np
from torchvision.models.detection import ssd300_vgg16, SSD300_VGG16_Weights, ssd

from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
import os
import cv2
import random
from dataset import ObjectDetectionDataset
# from dataset_helper import get_train_data_loader
from conf import *
import datetime
torch.manual_seed = 0

model_path1 = "models/FasterRcnn_V1_epoch-6_model.pth"
model_path2 = "models/FasterRcnn_V2_epoch-4_model.pth"
model_path3 = "models/SSD_epoch-8_model.pth"


In [52]:

def load_model1(num_classes=NUMBER_OF_CLASSES):
    model = fasterrcnn_resnet50_fpn(pretrained=False)
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
    state_dict= torch.load(model_path1)
    updated_state = {k.replace("module.", ""): v for k,v in state_dict.items()}
    model.load_state_dict(updated_state)
    print(f"MODEL from volume: {model_path1} is loaded successfuly")
    return model

def load_model2(num_classes=NUMBER_OF_CLASSES):
    model = fasterrcnn_resnet50_fpn_v2(pretrained=False)
    in_features = model.roi_heads.box_predictor.cls_score.in_features

    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
    state_dict= torch.load(model_path2)
    updated_state = {k.replace("module.", ""): v for k,v in state_dict.items()}
    model.load_state_dict(updated_state)
    print(f"MODEL from volume: {model_path2} is loaded successfuly")
    return model


def get_ssd_detection_model(num_classes=NUMBER_OF_CLASSES):
    ssd_model = ssd300_vgg16(weights=False)
    num_anchors = ssd_model.anchor_generator.num_anchors_per_location()
    out_channels = [512,1024,512,256,256,256]
    ssd_model.head = ssd.SSDHead(out_channels, num_anchors, num_classes+1)
    state_dict= torch.load(model_path3)
    updated_state = {k.replace("module.", ""): v for k,v in state_dict.items()}
    ssd_model.load_state_dict(updated_state)
    print(f"MODEL from volume: {model_path3} is loaded successfuly")

    return ssd_model


def save_test_img(img, target, prefix):
    # img = img.permute(2,0,1).cpu().numpy()  # Convert to (height, width, channels)
    img = img.cpu().numpy()  # Convert to (height, width, channels)

    # img = img.astype('uint8')
    # img = img
    # Draw bounding boxes on the image
    print(target)
    for box, label in zip(target['boxes'], target['labels']):
        x, y, w, h = box.tolist()
        x, y, w, h = int(x), int(y), int(w), int(h)
        # box_color = BOX_COLOR[label.item()]
        print(label)
        box_color = (255,255,255)
        cv2.rectangle(img, (x, y), (w, h), box_color, 2)

    # Save the image with bounding boxes
    if not os.path.exists(os.path.join(os.getcwd(), 'test_output')):
        os.makedirs(os.path.join(os.getcwd(), 'test_output'))
    # cv2.imshow(img)
    img_path = f"./test_output/output_image_{prefix}.png"
    cv2.imwrite(img_path, img)
    return img_path

def clean_targets(targets):
    cleaned_targets = {}
    # Filter out invalid boxes
    valid_boxes_mask = (targets['boxes'].sum(axis=1) > 2)    
    valid_labels_mask = (targets['labels'] != CLASSES_TO_IDX["background"])
    # valid_area_mask = (targets['area'] >= 1)

    # Combine all conditions using "&"
    final_mask = valid_boxes_mask & valid_labels_mask 
    # Apply the final mask
    for key, target_tensor in targets.items():
        if key == "idx":
            cleaned_targets[key] = target_tensor
            continue
        cleaned_targets[key] = target_tensor[final_mask]
    return cleaned_targets


def inference_filter_prediction(outputs, iou_threshold=0.25, confidence_threshold=0.50):
    cleaned_output = []
    for predicted_dict in outputs:
        mask = predicted_dict['scores'] >= confidence_threshold
        filtered_detections = {k: v[mask] for k, v in predicted_dict.items()}
        if len(filtered_detections['boxes'] != 0):
            nms_indices = nms(
                filtered_detections['boxes'],
                filtered_detections['scores'],
                iou_threshold
            )
            print("Before nms: ", len(filtered_detections["boxes"]))
            filtered_detections = {k: v[nms_indices] for k, v in filtered_detections.items()}
            print("After nms: ", len(filtered_detections["boxes"]))
        cleaned_output.append(filtered_detections)
    return cleaned_output


def get_images(image_path):
        # reading the images and converting them to correct size and color
        original_image = cv2.imread(image_path)
        grayscale = to_grayscale(original_image)
        grayscale = normalize_image(grayscale)
        grayscale = torch.from_numpy(grayscale).float()
        grayscale = grayscale.unsqueeze(0)
        
        return grayscale, torch.from_numpy(original_image)

def to_grayscale(image):	
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    return gray

def normalize_image(img):
    return img / 255


def train(model, train_loader, optimizer, epoch):
    device = torch.device("cpu")
    model.train()
    for batch_idx, (data, targets) in enumerate(train_loader, 1):
        data = list(image.to(device) for image in data)
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
        optimizer.zero_grad()
        output = model(data, targets)
        print(f"=====[ epoch {epoch} batch {batch_idx}  output of the model: {output}")

        loss = output["loss_classifier"]
        loss.backward()
        optimizer.step()

def run(model, train_loader):
    epochs = 2
    optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.5)
    scheduler = StepLR(optimizer, step_size=10, gamma=0.1)

    for epoch in range(1, epochs + 1):
        train(model, train_loader, optimizer, epoch)
        scheduler.step()



In [53]:
model1 = load_model1()
model2 = load_model2()
model3 = get_ssd_detection_model()

model1.eval()
model2.eval()
model3.eval()

original_image_sizes = []

MODEL from volume: models/FasterRcnn_V1_epoch-6_model.pth is loaded successfuly
MODEL from volume: models/FasterRcnn_V2_epoch-4_model.pth is loaded successfuly
MODEL from volume: models/SSD_epoch-8_model.pth is loaded successfuly


In [54]:
image_dir = "./real_images/simple.jpg"
grayscale, image = get_images(image_dir)
grayscale = grayscale.unsqueeze(0)
inputs = grayscale

timer_model1 = datetime.datetime.now()
outputs1 = model1(inputs)
timer_model1 = datetime.datetime.now() - timer_model1 

# print(f"outputs1: {outputs1}")
timer_model2 = datetime.datetime.now()
outputs2 = model2(inputs)
timer_model2 = datetime.datetime.now() - timer_model2

timer_model3 = datetime.datetime.now()
outputs3 = model3(inputs)
timer_model3 = datetime.datetime.now() - timer_model3


# print(f"outputs1: {outputs1}")
print("Model1 filtering: ")
outputs1 = inference_filter_prediction(outputs1)
print("Model2 filtering: ")
outputs2 = inference_filter_prediction(outputs2)
print("Model3 filtering: ")
outputs3 = inference_filter_prediction(outputs3)
print(f"Time\nModel1 (Rcnn_V1): {timer_model1}\nModel2 (Rcnn_V2): {timer_model2}\nModel3 (SSD): {timer_model3}")

it = random.randint(0, 1000)
prefix=f"detections-{it}"
print(prefix)
print(outputs3)

for output1, output2, output3 in zip(outputs1, outputs2,outputs3):
    save_test_img(torch.clone(image), output1, f"model1_{prefix}")
    save_test_img(torch.clone(image), output2, f"model2_{prefix}")
    save_test_img(image, output3, f"model3_{prefix}")





Model1 filtering: 
Before nms:  31
After nms:  20
Model2 filtering: 
Before nms:  37
After nms:  25
Model3 filtering: 
Before nms:  200
After nms:  53
Time
Model1 (Rcnn_V1): 0:00:00.913415
Model2 (Rcnn_V2): 0:00:02.120378
Model3 (SSD): 0:00:00.224682
detections-203
[{'boxes': tensor([[ 394.0913,  224.1995,  785.0116,  693.4707],
        [ 600.5184,  383.9464,  960.0000, 1280.0001],
        [ 410.5071,  440.4419,  714.4785, 1280.0001],
        [ 646.6525,    0.0000,  960.0000,  312.9229],
        [  13.4017,  351.1624,  385.5074, 1280.0001],
        [ 104.0245,  144.2947,  476.9404,  536.2286],
        [ 230.7604,    0.0000,  617.9042,  316.6730],
        [   0.0000,  645.7056,  127.3112,  840.1622],
        [ 521.5580,  801.0456,  731.2052, 1004.5587],
        [   0.0000,  513.8107,  131.2592,  701.6232],
        [ 643.7365,  656.8625,  865.9625,  852.7081],
        [ 157.9126,  647.4512,  347.2741,  822.4556],
        [   0.0000,    0.0000,  194.6432,  353.9010],
        [   0.0000,  